## A. Import Intervals and Other Code

* The first step is to import all the code required for the Notebook
* **`arrow/run`** or **`Shift + Enter`** in the following cell:

In [226]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os

MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)
else:
    print(MYDIR, "folder already exists.")
    
MUSDIR = ("Music_Files")
CHECK_FOLDER = os.path.isdir(MUSDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MUSDIR)
    print("created folder : ", MUSDIR)

else:
    print(MUSDIR, "folder already exists.")

saved_csv folder already exists.
Music_Files folder already exists.


## B. Importing Pieces

### B.1 Import a Single Piece and Check Metadata for Title and Composer

- Here you will want to select the appropriate 'prefix' that identifies the location of your file.
- `'Music_Files/'` is for files in the local notebook; `'https://crimproject.org/mei/'` is for the files on CRIM.
- Then provide the full name (and extension) of your music file, such as `'CRIM_Model_0038.mei'`

In [227]:
# Select a prefix:
# prefix = 'Music_Files/'
prefix = 'https://crimproject.org/mei/' 


# Add your filename here
mei_file = 'CRIM_Model_0008.mei'

# These join the strings and import the piece
url = prefix + mei_file
piece = importScore(url)

print(piece.metadata)

{'title': 'Ave Maria', 'composer': 'Josquin Des Prés', 'date': 1502}


## P Types with Details of Start/Stop

In [228]:
# get the P-types, ngrams as series, and the notes with detail index
# must pass the same "n" to mel and ptypes
ngram_length = 4

p_types = piece.presentationTypes(melodic_ngram_length=ngram_length)
# get the ngrams as a series
notes = piece.notes()
notes_det = piece.detailIndex(notes, offset=True)
mel = piece.melodic(end=False)
ngrams = piece.ngrams(df=mel, n=ngram_length, offsets='both')
ngrams_series = ngrams.stack()

# notes
notes_det = notes_det.reset_index(level=['Measure', 'Beat'])
notes_det = notes_det[['Measure', 'Beat']]
notes_det.index.names = ['Offset']

#voice dict
_dict = piece._getPartNumberDict()


In [229]:
# the function
def ptype_ema_table(row, ngrams_series, notes_det, _dict):
    offsets = row["Offsets"]
    voices = row["Voices"]
    pairs = list(zip(offsets, voices))

    ng_list = []
    for pair in pairs:
        offset = pair[0]
        voice = pair[1]
        ngram_coordinates = ngrams_series[(ngrams_series.index.get_level_values(0) == offset) & (ngrams_series.index.get_level_values(2) == voice)]
        ngram_coordinates = ngram_coordinates.reset_index()
        ngram_coordinates = ngram_coordinates.rename(columns={'level_2' : 'Voice'})
        ng_list.append(ngram_coordinates)
    ng_df = pd.concat(ng_list)
    ng_df_first = ng_df.set_index('First')
    ng_df_last = ng_df.set_index('Last')
    merged_df = ng_df_first.merge(notes_det, left_index=True, right_index=True, how='left')
    merged_df.rename(columns={"Measure" : "FirstMeasure", "Beat" : "FirstBeat"}, inplace=True)
    merged_df.reset_index(inplace=True)
    merged_df2 = ng_df_last.merge(notes_det, left_index=True, right_index=True, how='left')
    merged_df2.rename(columns={"Measure" : "LastMeasure", "Beat" : "LastBeat"}, inplace=True)
    merged_df2.reset_index(inplace=True)
    cols_to_use = merged_df2.columns.difference(merged_df.columns)
    final = pd.merge(merged_df, merged_df2[cols_to_use], left_index=True, right_index=True, how='outer')
    final['PartNumber'] = final['Voice'].map(_dict)
    final.drop(columns=['Voice'], inplace=True)
    final.rename(columns={0 : 'Ngram'}, inplace=True)
    final.set_index(["First", "Last"], inplace=True)
    return final

In [230]:
p_types['result'] = p_types.apply(lambda row: ptype_ema_table(row, ngrams_series, notes_det, _dict), axis=1)
sample = p_types.iloc[0]['result']
sample



,,Ngram,FirstMeasure,FirstBeat,LastBeat,LastMeasure,PartNumber
First,Last,,,,,,
0.0,20.0,"(P4, P1, M2, M2)",1,1.0,3.0,3,1
16.0,36.0,"(P4, P1, M2, M2)",3,1.0,3.0,5,2
32.0,52.0,"(P4, P1, M2, M2)",5,1.0,3.0,7,3
48.0,68.0,"(P4, P1, M2, M2)",7,1.0,3.0,9,4


In [231]:
piece.emaAddresses(df=sample)

First  Last
0.0    20.0    1-3/Ngram+FirstMeasure+FirstBeat+LastBeat+Last...
16.0   36.0    3-5/Ngram+FirstMeasure+FirstBeat+LastBeat+Last...
32.0   52.0    5-7/Ngram+FirstMeasure+FirstBeat+LastBeat+Last...
48.0   68.0    7-9/Ngram+FirstMeasure+FirstBeat+LastBeat+Last...
dtype: object

# HR with First/Last Coordinates

In [232]:
# get the P-types, ngrams as series, and the notes with detail index
# must pass the same "n" to mel and ptypes
ngram_length = 4

# hr = piece.homorhythm(ngram_length=ngram_length)
hr = piece.homorhythm(ngram_length = ngram_length).reset_index()
hr = hr.rename(columns={"Measure" : "FirstMeasure", "Beat": "FirstBeat", "hr_voices" : "Voices"})
# get the ngrams as a series
notes = piece.notes()
notes_det = piece.detailIndex(notes, offset=True)
mel = piece.melodic(end=False)
ngrams = piece.ngrams(df=mel, n=ngram_length, offsets='both')
ngrams_series = ngrams.stack()

# notes
notes_det = notes_det.reset_index(level=['Measure', 'Beat'])
notes_det = notes_det[['Measure', 'Beat']]
notes_det.index.names = ['Offset']

#voice dict
_dict = piece._getPartNumberDict()


In [233]:
hr.head(2)

,FirstMeasure,FirstBeat,Offset,active_voices,number_dur_ngrams,Voices,[Superius],Altus,Tenor,Bassus,syllable_set,count_lyr_ngrams,active_syll_voices,voice_match
0,31,3.0,244.0,2.0,1.0,"[[Superius], Altus]","(A, ve, cae, lo)","(A, ve, cae, lo)",NaN,NaN,"[(A, ve, cae, lo), (A, ve, cae, lo)]",1.0,2.0,True
1,35,3.0,276.0,2.0,1.0,"[Tenor, Bassus]",NaN,NaN,"(A, ve, cae, lo)","(A, ve, cae, lo)","[(A, ve, cae, lo), (A, ve, cae, lo)]",1.0,2.0,True


In [234]:
# the function
def hr_ema_table(row, ngrams_series, notes_det, _dict):
    offset = row["Offset"]
    voices = row["Voices"]
    pairs = []
    for voice in voices:
        pair = (offset, voice)
        pairs.append(pair)

    ng_list = []
    for pair in pairs:
        offset = pair[0]
        voice = pair[1]
        ngram_coordinates = ngrams_series[(ngrams_series.index.get_level_values(0) == offset) & (ngrams_series.index.get_level_values(2) == voice)]
        ngram_coordinates = ngram_coordinates.reset_index()
        ngram_coordinates = ngram_coordinates.rename(columns={'level_2' : 'Voice'})
        ng_list.append(ngram_coordinates)
    ng_df = pd.concat(ng_list)
    ng_df_first = ng_df.set_index('First')
    ng_df_last = ng_df.set_index('Last')

    merged_df = ng_df_first.merge(notes_det, left_index=True, right_index=True, how='left')
    merged_df.rename(columns={"Measure" : "FirstMeasure", "Beat" : "FirstBeat"}, inplace=True)
    merged_df.reset_index(inplace=True)
    merged_df2 = ng_df_last.merge(notes_det, left_index=True, right_index=True, how='left')
    merged_df2.rename(columns={"Measure" : "LastMeasure", "Beat" : "LastBeat"}, inplace=True)
    merged_df2.reset_index(inplace=True)
    cols_to_use = merged_df2.columns.difference(merged_df.columns)
    final = pd.merge(merged_df, merged_df2[cols_to_use], left_index=True, right_index=True, how='outer')
    final.reset_index(inplace=True)
    final.rename(columns={0 : "ngram"}, inplace=True)
    final['PartNumber'] = final['Voice'].map(_dict)
    final = final[["First", "Last", "PartNumber", "FirstMeasure", "FirstBeat", "LastMeasure", "LastBeat"]]
    final.set_index(["First", "Last"], inplace=True)
    

    return final

In [235]:
hr['result'] = hr.apply(lambda row: hr_ema_table(row, ngrams_series, notes_det, _dict), axis=1)

# sample
sample = hr.iloc[0]['result']
sample



PartNumber  FirstMeasure  FirstBeat  LastMeasure  LastBeat
First Last                                                            
244.0 260.0          1            31        3.0           33       3.0
      260.0          2            31        3.0           33       3.0

In [236]:
piece.emaAddresses(df=sample)

First  Last 
244.0  260.0    31-33/PartNumber+FirstMeasure+FirstBeat+LastMe...
       260.0    31-33/PartNumber+FirstMeasure+FirstBeat+LastMe...
dtype: object

# Cadences with First/Last

In [237]:
# for cadences
# get the following first
# must pass the same "n" to mel.  It's the default for cvfs anyway
ngram_length = 3

# get the ngrams as a series
notes = piece.notes()
notes_det = piece.detailIndex(notes, offset=True)
mel = piece.melodic(end=False)
ngrams = piece.ngrams(df=mel, n=ngram_length, offsets='both')

# notes
notes_det = notes_det.reset_index(level=['Measure', 'Beat'])
notes_det = notes_det[['Measure', 'Beat']]
notes_det.index.names = ['Offset']

# voice numbers
_dict = piece._getPartNumberDict()

In [238]:
cvfs = piece.cvfs(keep_keys=True)
cvfs = cvfs[list(_dict.keys())]
cvfs.index.names = ['Offset']
cvfs.head(2)

,[Superius],Altus,Tenor,Bassus
Offset,,,,
188.0,NaN,T,C,b
232.0,C,u,T,b


In [239]:
# LAST offsets of ngrams
ngrams_last = ngrams.reset_index(level=0)
ngrams_last.index.rename('Offset', inplace=True)
ngrams_last = ngrams_last.drop(['First'], axis=1)

# FIRST offsets of ngrams

ngrams_first = ngrams.reset_index(level=1)
ngrams_first.index.rename('Offset', inplace=True)
ngrams_first = ngrams_first.drop(['Last'], axis=1)

#now merging
merged_df = ngrams_last.merge(notes_det, left_index=True, right_index=True, how='left')
merged_df.rename(columns={"Measure" : "FirstMeasure", "Beat" : "FirstBeat"}, inplace=True)
merged_df.reset_index(inplace=True)
merged_df.rename(columns={"Offset" : "Last"}, inplace=True)
merged_df2 = ngrams_first.merge(notes_det, left_index=True, right_index=True, how='left')
merged_df2.rename(columns={"Measure" : "LastMeasure", "Beat" : "LastBeat"}, inplace=True)
merged_df2.reset_index(inplace=True)
merged_df2.rename(columns={"Offset" : "First"}, inplace=True)
cols_to_use = merged_df2.columns.difference(merged_df.columns)
merged_ng_locations = pd.merge(merged_df, merged_df2[cols_to_use], left_index=True, right_index=True, how='outer')
merged_ng_locations = merged_ng_locations.set_index(["First", "Last"])
merged_ng_locations = merged_ng_locations[['FirstMeasure', 'FirstBeat', 'LastBeat', 'LastMeasure']]
merged_ng_locations.reset_index(level=0, inplace=True)
merged_ng_locations.index.rename('Offset', inplace=True)
cvfs_with_locations= cvfs.merge(merged_ng_locations, left_index=True, right_index=True, how='left')
cvfs_with_locations.rename(columns=_dict, inplace=True)
cvfs_with_locations.reset_index(inplace=True)
cvfs_with_locations.rename(columns={"Offset" : "Last"}, inplace=True)
cvfs_with_locations.set_index(["First", "Last"], inplace=True)
cvfs_with_locations.drop_duplicates(inplace=True)
cvfs_with_locations



,,1,2,3,4,FirstMeasure,FirstBeat,LastBeat,LastMeasure
First,Last,,,,,,,,
178.0,188.0,NaN,T,C,b,24,3.0,2.0,23
222.0,232.0,C,u,T,b,30,1.0,4.0,28
224.0,232.0,C,u,T,b,30,1.0,1.0,29
262.0,272.0,C,T,NaN,NaN,35,1.0,4.0,33
265.0,272.0,C,T,NaN,NaN,35,1.0,1.5,34
294.0,304.0,NaN,A,C,T,39,1.0,4.0,37
297.0,304.0,NaN,A,C,T,39,1.0,1.5,38
408.0,416.0,C,NaN,T,B,53,1.0,1.0,52
409.0,416.0,C,NaN,T,B,53,1.0,1.5,52


In [240]:
piece.emaAddresses(df=cvfs_with_locations)

First   Last  
178.0   188.0     23-24/2+3+4+FirstMeasure+FirstBeat+LastBeat+La...
222.0   232.0     28-30/1+2+3+4+FirstMeasure+FirstBeat+LastBeat+...
224.0   232.0     29-30/1+2+3+4+FirstMeasure+FirstBeat+LastBeat+...
262.0   272.0     33-35/1+2+FirstMeasure+FirstBeat+LastBeat+Last...
265.0   272.0     34-35/1+2+FirstMeasure+FirstBeat+LastBeat+Last...
294.0   304.0     37-39/2+3+4+FirstMeasure+FirstBeat+LastBeat+La...
297.0   304.0     38-39/2+3+4+FirstMeasure+FirstBeat+LastBeat+La...
408.0   416.0     52-53/1+3+4+FirstMeasure+FirstBeat+LastBeat+La...
409.0   416.0     52-53/1+3+4+FirstMeasure+FirstBeat+LastBeat+La...
464.0   472.0     59-60/1+2+3+FirstMeasure+FirstBeat+LastBeat+La...
465.0   472.0     59-60/1+2+3+FirstMeasure+FirstBeat+LastBeat+La...
504.0   512.0     64-65/1+3+4+FirstMeasure+FirstBeat+LastBeat+La...
505.0   512.0     64-65/1+3+4+FirstMeasure+FirstBeat+LastBeat+La...
600.0   608.0     76-77/1+3+4+FirstMeasure+FirstBeat+LastBeat+La...
601.0   608.0     76-77/1+3+4+Fir